In [1]:
import pandas as pd

In [2]:
data = 'data/data.csv'

# other csv files are incomplete, so I will focus on 'data.csv'

In [93]:
def wrangle_data(file):
    df = pd.read_csv(file)
    artist = []
    for name in df['artists']:
        name = name.replace('\'', '')
        name = name.replace('[', '')
        name = name.replace(']', '')
        name = name.replace('\"', '')
        artist.append(name)
    df['artists'] = artist
    df = df.drop(columns='release_date')
    df['url'] = 'https://open.spotify.com/track/' + df['id']
    df = df[['id', 'name', 'artists', 'url', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo',
       'valence', 'year']]
    df = df.drop_duplicates()
    return df


In [95]:
test.shape

(172230, 19)

In [94]:
test = wrangle_data(data)
test.tail()

,id,name,artists,url,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
174379,45XnLMuqf3vRfskEAMUeCH,A Little More,Alessia Cara,https://open.spotify.com/track/45XnLMuqf3vRfsk...,0.795,0.429,144720,0.211,0,0.0,4,0.196,-11.665,1,0,0.0360,94.710,0.228,2021
174381,4pPFI9jsguIh3wC7Otoyy8,A Little More,Alessia Cara,https://open.spotify.com/track/4pPFI9jsguIh3wC...,0.795,0.429,144720,0.211,0,0.0,4,0.196,-11.665,1,0,0.0360,94.710,0.228,2021
174383,52YtxLVUyvtiGPxwwxayHZ,A Little More,Alessia Cara,https://open.spotify.com/track/52YtxLVUyvtiGPx...,0.795,0.429,144720,0.211,0,0.0,4,0.196,-11.665,1,0,0.0360,94.710,0.228,2021
174385,7tue2Wemjd0FZzRtDrQFZd,A Little More,Alessia Cara,https://open.spotify.com/track/7tue2Wemjd0FZzR...,0.795,0.429,144720,0.211,0,0.0,4,0.196,-11.665,1,0,0.0360,94.710,0.228,2021
174387,1gcyHQpBQ1lfXGdhZmWrHP,champagne problems,Taylor Swift,https://open.spotify.com/track/1gcyHQpBQ1lfXGd...,0.920,0.462,244000,0.240,1,0.0,0,0.113,-12.077,1,69,0.0377,171.319,0.320,2021


In [65]:
target = ['id', 'name', 'artists', 'url']
X = test.drop(columns=target)
y = test[target]

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42) 

In [77]:
assert int(len(X)*.8) == len(X_train), 'check test set'
assert len(X)-int(len(X)*.8) == len(X_val), 'check validation set'
assert int(len(X)*.8) == len(y_train), 'check test set'
assert len(X)-int(len(X)*.8) == len(y_val), 'check validation set'

In [79]:
# vectorize 
X.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
0,0.991000,0.598,168333,0.224,0,0.000522,5,0.3790,-12.628,0,12,0.0936,149.976,0.6340,1920
1,0.643000,0.852,150200,0.517,0,0.026400,5,0.0809,-7.261,0,7,0.0534,86.889,0.9500,1920
2,0.993000,0.647,163827,0.186,0,0.000018,0,0.5190,-12.098,1,4,0.1740,97.600,0.6890,1920
3,0.000173,0.730,422087,0.798,0,0.801000,2,0.1280,-7.311,1,17,0.0425,127.997,0.0422,1920
4,0.295000,0.704,165224,0.707,1,0.000246,10,0.4020,-6.036,0,2,0.0768,122.076,0.2990,1920


In [83]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors()
model.fit(X_train, y_train)

NearestNeighbors()

In [91]:
model.radius_neighbors(X_val)

(array([array([], dtype=float64), array([], dtype=float64),
        array([], dtype=float64), ..., array([0.]),
        array([], dtype=float64), array([], dtype=float64)], dtype=object),
 array([array([], dtype=int64), array([], dtype=int64),
        array([], dtype=int64), ..., array([61913]),
        array([], dtype=int64), array([], dtype=int64)], dtype=object))

In [98]:
nn = NearestNeighbors(algorithm='brute', leaf_size =12, n_neighbors=15, n_jobs=-1)
nn.fit(X)

# sample a song(index) from X to use as our query point 
input = 0
input_index=input

# vectorize 
data_vect = [X.iloc[input_index].values]
neigh_dist, neigh_indices = nn.kneighbors(data_vect)
indexs = neigh_indices.flat[0:12].tolist()

output = y.iloc[indexs]

In [99]:
output[0]

,id,name,artists,url
0,0cS0A1fUEUd1EW3FcF8AEI,Keep A Song In Your Soul,Mamie Smith,https://open.spotify.com/track/0cS0A1fUEUd1EW3...
127187,74vda4pNQoNuh2DBxlssnb,Şu Kızı Bana Vermiyorlar,Makbule Enver Hanım,https://open.spotify.com/track/74vda4pNQoNuh2D...
59054,56HmximYQWTjpghFQOGqRy,De a Traicion - Remasterizado,"Francisco Canaro, Charlo",https://open.spotify.com/track/56HmximYQWTjpgh...
1696,0824IiWIQJCw9GP8RKjU6o,De a Traicion,"Francisco Canaro, Charlo",https://open.spotify.com/track/0824IiWIQJCw9GP...
60455,4BspizHHJl4dY03BD8FooZ,O mia bella Napoli,Titi Botez,https://open.spotify.com/track/4BspizHHJl4dY03...
21271,2vebVasXSfdp7SAOzuaXvQ,Sos Un Hombre - Instrumental (Remasterizado),Francisco Canaro,https://open.spotify.com/track/2vebVasXSfdp7SA...
1314,0NfIlp9ZOS0wHLqtpExbti,¿Cuando Volverás? - Remasterizado,"Francisco Canaro, Agustín Irusta",https://open.spotify.com/track/0NfIlp9ZOS0wHLq...
94593,4LWO3ULGwCgcCW8ypKt7zN,Consejo de Amigo,Pedro Maffia,https://open.spotify.com/track/4LWO3ULGwCgcCW8...
6426,5hU6H5pEbwOa0fLJKLyMwJ,It Could Happen To You - Remastered,Nat King Cole,https://open.spotify.com/track/5hU6H5pEbwOa0fL...
63,1QI5WTvnpNV5vJJQU4DPX1,El Baccarat - Remasterizado,Francisco Canaro,https://open.spotify.com/track/1QI5WTvnpNV5vJJ...
